In [1]:
import numpy as np

In [81]:
# Generate an adjacency matrix for a small network
n=3
A = np.zeros((n,n), dtype=int)
for i in range(n):
    for j in range(n):
        A[i,j] = np.random.choice([0,1])

print(A)

[[1 1 0]
 [1 1 1]
 [1 0 1]]


We're gonna try a convention as follows:
  * The entry 0 + 0j corresponds to 0
  * The entry 1 + 0j corresponds to ,
  * The entry 0 + 1j corresponds to a message variable $m_\alpha$ for $\alpha \geq 1$

We will have a counter for $\alpha$, and as the simulation progresses, we'll populate a dictionary whose keys are values of $\alpha.$ Every time a message goes extinct, we'll increment the counter and re-inject 0 + 1j. 

In [11]:
# TEST1: The zero vector goes to the zero vector
x = np.array([complex("j") for i in range(n)])

print(np.matmul(A,x))

[0.+1.j 0.+1.j 0.+1.j]


Below, we make a first pass at building the simulation.

In [68]:
# Gives a random binary vector
def get_b(n):
    return np.array( [np.random.choice([complex("0"),complex("1")]) for i in range(n)] )

# In place, randomly selects an entry of 1 in b and replaces it with 1j.
def inject_tracker(b):
    support = [i for i in range(b.shape[0]) if b[i,] ]
    node = np.random.choice(support)
    b[node,] = complex("j")

# The collision function. The input is a vector y = Ax + b
def f(y):
    # We multiply each entry of y by the indicator that there is only one message at the corresponding node 
    return np.array( [ y[i,]*(y[i,].real + y[i,].imag == 1) for i in range(y.shape[0]) ] )


In [76]:
# Initialize external parameters
survival_times = dict()
alpha = 0
need_injection = True
T = 10

# Initialize dynamical parameters (t=0)
x = get_b(n)

# This is the main loop
for t in range(0,T+1):
    # Get b_t
    b = get_b(n)

    # If it is possible and necessary to inject a tracker, do so, and update values accordingly
    if (np.sum(b) > 0) and need_injection:
        inject_tracker(b)
        need_injection = False
        alpha += 1
        survival_times[alpha] = 1
    
    # Calculate the next state
    x = f(np.matmul(A,x) + b)

    # Check if message alpha has survived, and if so, update accordingly
    if complex("j") in x:
        survival_times[alpha] += 1
    else:
        need_injection = True

In [77]:
survival_times

{1: 1, 2: 3, 3: 1, 4: 3, 5: 2}

Alright, this looks pretty good! Still, I'd like to do a run and then compare it to what I do by hand. So, I'll modify the above code to give a print out with which I can follow along.

In [82]:
# Initialize external parameters
survival_times = dict()
alpha = 0
need_injection = True
T = 10

# Initialize dynamical parameters (t=0)
x = get_b(n)

# This is the main loop
for t in range(0,T+1):
    print("t = " + str(t))
    print("x = ")
    print(x)

    # Get b_t
    b = get_b(n)

    # If it is possible and necessary to inject a tracker, do so, and update values accordingly
    if (np.sum(b) > 0) and need_injection:
        inject_tracker(b)
        need_injection = False
        alpha += 1
        survival_times[alpha] = 1
    print("b = ")
    print(b)
    print()
    
    # Calculate the next state
    x = f(np.matmul(A,x) + b)

    # Check if message alpha has survived, and if so, update accordingly
    if complex("j") in x:
        survival_times[alpha] += 1
    else:
        need_injection = True

t = 0
x = 
[0.+0.j 0.+0.j 0.+0.j]
b = 
[0.+1.j 0.+0.j 0.+0.j]

t = 1
x = 
[0.+1.j 0.+0.j 0.+0.j]
b = 
[0.+0.j 0.+0.j 0.+0.j]

t = 2
x = 
[0.+1.j 0.+1.j 0.+1.j]
b = 
[1.+0.j 1.+0.j 0.+0.j]

t = 3
x = 
[0.+0.j 0.+0.j 0.+0.j]
b = 
[0.+0.j 0.+1.j 1.+0.j]

t = 4
x = 
[0.+0.j 0.+1.j 1.+0.j]
b = 
[1.+0.j 1.+0.j 0.+0.j]

t = 5
x = 
[0.+0.j 0.+0.j 1.+0.j]
b = 
[0.+1.j 1.+0.j 0.+0.j]

t = 6
x = 
[0.+1.j 0.+0.j 1.+0.j]
b = 
[0.+0.j 1.+0.j 0.+0.j]

t = 7
x = 
[0.+1.j 0.+0.j 0.+0.j]
b = 
[1.+0.j 0.+0.j 1.+0.j]

t = 8
x = 
[0.+0.j 0.+1.j 0.+0.j]
b = 
[1.+0.j 1.+0.j 1.+0.j]

t = 9
x = 
[0.+0.j 0.+0.j 1.+0.j]
b = 
[0.+0.j 0.+1.j 0.+0.j]

t = 10
x = 
[0.+0.j 0.+0.j 1.+0.j]
b = 
[1.+0.j 0.+1.j 0.+0.j]



In [83]:
survival_times

{1: 3, 2: 2, 3: 4, 4: 1, 5: 1}